In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

이온채널 관련 자료: https://newsight.tistory.com/189
![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile28.uf.tistory.com%2Fimage%2F2559234B552FBE91177A12)



patch clamp 원리:  https://m.blog.naver.com/PostView.nhn?blogId=wjddnjsrbs97&logNo=220908487841&proxyReferer=https%3A%2F%2Fwww.google.com%2F

![](http://www.ibric.org/upload/geditor/201711/0.44270300_1511918463.jpg)

: patch clamp는 유리피펫으로 세포의 전류를 측정법   
  whole-cell 모드에서 전류를 측정함 (이번 대회 데이터는 논문에 patch-clamp로 측정된 data로 나옴)

이온통로 patch clamp :  https://ekja.org/upload/pdf/kjae-42-273.pdf


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


관련논문
https://www.nature.com/articles/s42003-019-0729-3


F1 Measure 관련:
https://sumniya.tistory.com/26

QWK:
https://www.kaggle.com/aroraaman/quadratic-kappa-metric-explained-in-5-simple-steps


In [ ]:
train = pd.read_csv('../input/liverpool-ion-switching/train.csv')

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train.open_channels.value_counts()

500만개의 데이터가 10개의 batch로 구성이 되어 있음

50초 동안 10khz 로 측정 => 500,000 개

10개 배치로 구성되어 각 배치별로 분석필요함

In [ ]:
plt.figure(figsize=(20,5)); res = 1000 #간격을 1000 만큼씩 띄어서 표시
x = range(0,train.shape[0],res)
y = train.signal[0::res]
plt.plot(x,y,'b',alpha=0.7)
for i in range(11): 
    plt.plot([i*500000,i*500000],[-5,12.5],'r')
for j in range(10): 
    plt.text(j*500000+200000,10,str(j+1),size=16)
plt.xlabel('Row',size=16); plt.ylabel('Signal & Ion channels',size=16); 
plt.title('Training Data Signal - 10 batches',size=20)

#plt.figure(figsize=(20,5))
y2 = train.open_channels[0::res]
plt.plot(x,y2,'r',alpha=0.3) #Ion 채널 수 표시

plt.show()

오픈된 이온채널의 수량과 차이는 있어보임

Drift : 2,7,8,9,10  (signal의 변화가 추세가 있음)  --> drift는 조금 더 공부필요 

---

1,2 : 오픈채널 1개, 낮은 빈도로 열림  1번 수평, 2번 초반 drift

---

3,7 : 오픈채널 1개, 높은 빈도로 열림  3번 수평, 7번 포물선  --> 3,7은 어떤 특성의 차이로 수평과 포물선으로 구분?

---

4,8 : 오픈채널 3개, 높은 빈도로 열림  4번 수평, 8번 포물선  --> 4,8은 어떤 특성의 차이가 수평과 포물선으로 구분?

---

6,9 : 오픈채널 4개, 높은 빈도로 열림  6번 수평, 9번 포물선  --> 6,9는 어떤 특성의 차이가 수평과 포물선으로 구분?

---

5,10: 오픈채널 10개, 높은 빈도로 열림, 5번 수평, 10번 포물선 --> 5,10는 어떤 특성의 차이가 수평과 포물선으로 구분?

데이터를 각 특성별로 다시 재구분

In [ ]:
train.corr()


signal값과 open_channels 간에는 0.8 정도의 상관관계임 (시간외에 feature가 signal뿐이니 당연하겠지만)

In [ ]:
#각 배치를 10개 데이터 리스트로 구분
batch = []
gap = 500000
batch.append([]) #dummy
for i in range(10):
    start = i*gap
    batch.append(train[start:start+gap])

In [ ]:
group1 = [batch[1],batch[2]]
group2 = [batch[3],batch[7]]
group3 = [batch[4],batch[8]]
group4 = [batch[6],batch[9]]
group5 = [batch[5],batch[10]]


In [ ]:
#plot작성용 helper 
def plot_batch(batch_no=0, start=0, end=500000):
    plt.figure(figsize=(20,5))
    x = range(end-start)
    y1 = batch[batch_no].signal[start:end]
    y2 = batch[batch_no].open_channels[start:end]
    plt.plot(x,y1)
    plt.plot(x,y2,'r',alpha=0.5)
    plt.legend()
    plt.show()


In [ ]:
plot_batch(6) #6번 배치를 출력

50만개는 너무 많아서 보이지 않음....

In [ ]:
plot_batch(6,0,100) # 6번 배치에서 0~100

signal의 변화에 따라서 채널수 변화가 보임

In [ ]:
plot_batch(6,40,60) #6번 배치에서 40~60

10번 배치 살펴보기: 10개 채널을 모두 오픈하는 상황이 많음, 비율은?

In [ ]:
plot_batch(10,0,500000)

In [ ]:
batch[10].describe()

In [ ]:
batch[10].open_channels.value_counts()

In [ ]:
plot_batch(1,100,20000) #1번 배치에서 100~200 

In [ ]:
plot_batch(1,15200,15400)

오픈 채널이 0 인 상황은, signal 값이 -2 이하가 유지될 때인가?

In [ ]:
train.describe()

In [ ]:
# 데이터 값 실수. 소수점 자리 설정
pd.options.display.float_format = '{:.2f}'.format

sig_mean = []
sig_std = []

for i in range(11):
    print('='*10, i , '='*40)
    df_desc = train[train.open_channels == i][['signal','open_channels']].describe().T[['count','mean','std','min','max']]
    sig_mean.append(df_desc['mean'][0] )
    sig_std.append(df_desc['std'][0])
    print(df_desc)

In [ ]:
import scipy.stats as stats

In [ ]:
x = np.linspace(-15, 15, 200) #x축 값, 200개 값을 -15에서 15사이에 균일간격으로 생성  

In [ ]:
legend = []

def norm (mu, var) :
    legend.append(str(i) + ": N(" + str(round(mu,3)) + ", " + str(round(var,3)) + ")")
    return stats.norm(mu, var).pdf(x)        

plt.figure(figsize=(20, 6))          # 플롯 사이즈 지정

for i in range(len(sig_mean)):
    plt.plot(x, norm(sig_mean[i], sig_std[i]))          # plot 추가       
    
plt.xlabel("Signal")                      # x축 레이블 지정
plt.ylabel("Open ION Channel")                      # y축 레이블 지정
plt.grid()                           # 플롯에 격자 보이기
plt.title("[ Signal distribution per open channel ]")     # 타이틀 표시
plt.legend(legend)                   # 범례 표시
plt.show()                           # 플롯 보이기


open channel수량 별로 signal값의 분포인데.... 각 채널 수량별로 다르긴 하지만 중첩구간도 많음 ㅠㅠ

In [ ]:

plt.figure(figsize=(20, 5))
sns.boxplot(x="open_channels", y="signal", data=train) #실용적, 2초 정도 걸림, violinplot (11초) 보다 빠름, stipplot (120초)
plt.show() 

box plot 이 현재 10개 채널을 비교해 보기에는 더 적합해 보임


Seaborn plot 참고용
[https://datascienceschool.net/view-notebook/4c2d5ff1caab4b21a708cc662137bc65/](https://datascienceschool.net/view-notebook/4c2d5ff1caab4b21a708cc662137bc65/)



In [ ]:
plt.figure(figsize=(20,5))
sns.violinplot(x="open_channels",y="signal",data=train)
plt.show() #violinplot은 나오는데 boxplot보다 시간이 더 오래걸림, 11초 이상

채널수가 많아서 10개를 표현하면 너무 폭이 적어서 분포가 눈에 정확히 보이지는 않음, 특성이 있는 분류별로 나눠서 보는게 어떨지....



In [ ]:
#120초 정도 걸려서 보기 힘들고, 점이 너무 많아서 구분이 되지 않음 
#plt.figure(figsize=(20,5))
#sns.stripplot(x='open_channels', y='signal', data=train)
#plt.show()

In [ ]:
#rolling 사용법 
# rolling(window=데이터몇개를 묶어서 rolling 처리할지, min_periods= window사이즈가 되지 않아도 처리하는 최소크기, 
#       center=기본 false로, true이면 window의 중간 중심으로 앞뒤 window/2씩 rolling 됨 )

plt.figure(figsize=(20,5))
batch[2].signal.rolling(500).mean().plot()
plt.show()

초기 전체데이터를 각 배치를 5개로 구분했음, 직선형태와 포물선의 차이 확인하기


ion channel 1개 열림: 직선 3번, 포물선 7번 체크

In [ ]:
plot_batch(3,0,500000)

In [ ]:
plot_batch(7,0,500000)

채널이 1) 열릴 때 2) 닫힐 때 3)유지될 때 의 특성은 어떻게 나타날까?

1) 열릴 때: 3,7은 열린게 주로 유지된 상태 임, 열리기 직전에 어느정도의 signal이 영향을 주는지? 

2) 닫힐 때: 닫히기 직전에 어느정도의 signal이 영향을 주는지? 

3) 유지될 때: 유지되는 패턴은?



In [ ]:
plot_batch(3,650,850)

In [ ]:
plot_batch(7,650,850)

In [ ]:
#plot작성용 helper, rolling추가 
def plot_rolling(batch_no=0, start=0, end=500000, window=10, rtype='mean'):
    plt.figure(figsize=(20,5))
    x = range(end-start)
    y1 = batch[batch_no].signal[start:end]
    y2 = batch[batch_no].open_channels[start:end]
    if rtype == 'min':
        y3 = batch[batch_no].signal[start:end].rolling(window=window,min_periods=1,center=False).min()
    elif rtype == 'max':
        y3 = batch[batch_no].signal[start:end].rolling(window=window,min_periods=1,center=False).max()
    else:
        y3 = batch[batch_no].signal[start:end].rolling(window=window,min_periods=1,center=False).mean()
    
    plt.plot(x,y1)
    plt.plot(x,y2,'y',alpha=0.5)
    plt.plot(x,y3,'r',alpha=0.5)
    plt.legend()
    plt.grid()
    plt.show()


In [ ]:
plot_rolling(7,650,750, window=10)

In [ ]:
plot_rolling(7,650,750, window=3)

In [ ]:
plot_rolling(7,650,750, window=2, rtype='min')

아직 잘 모르겠음.... 닫힐 때는 어떤 상관관계가 있는지 눈에 확실하게 보이지는 않음

In [ ]:
batch[1].head(3)

In [ ]:
sns.lineplot(x='time',y='signal',data=batch[1][100:200])


In [ ]:
np.round?

<코드작성시 참조>

코드 자동완성: Tab

코드 설명: Tab + Shift (1번 누르면 상단: 쥬피터에서는 정상, 4번 눌러야 하단) , ? 방식이 나음

코드의 파란색 클릭시 관련 함수 사용한 다른 노트북들 표시

Markov Chain 관련

On Markov Chains and the Competition Data (대회참가자)
[https://www.kaggle.com/friedchips/on-markov-chains-and-the-competition-data](https://www.kaggle.com/friedchips/on-markov-chains-and-the-competition-data)


Markov Chain
https://rfriend.tistory.com/184

[https://towardsdatascience.com/introduction-to-markov-chains-50da3645a50d](https://towardsdatascience.com/introduction-to-markov-chains-50da3645a50d)


선형대수 설명
https://bskyvision.com/59


아이겐벡터
[https://rfriend.tistory.com/181](https://rfriend.tistory.com/181)


PCA
https://ratsgo.github.io/machine%20learning/2017/04/24/PCA/


transit probability 를 아래 함수를 사용해서 계산
T = P(next 오픈채널수 | current 오픈채널) 

In [ ]:
def transit_matrix(data):
    channel_range = np.unique(data) #채널오픈 수량 중복제거 추출, 0,1,2,.... 10
    channel_bins = np.append(channel_range, 11) # bins 는 0,1,2,3,11 의미는 0~1, 1~2, 2~3, 3~11 의 구간임
    data_next = np.roll(data, -1)    # 1칸 앞으로 말기, 뒤에 가서 붙음
    matrix = []
    for i in channel_range:
        current_row = np.histogram(data_next[data == i], bins=channel_bins)[0]    #2개 리턴되는데, 첫번째가 각 bind 에 들어있는 수량 
        current_row = current_row / np.sum(current_row)     # i=2 일 경우,  현재 오픈된 채널별 갯수  / 직전 오픈채널이 2인 것의 총 갯수
        matrix.append(current_row) 
    return np.array(matrix)  # 행: 직전 오픈채널 값, 열: 현재 오픈채널 값 , 데이터: 해당 행,열 기준의 대상 수량

In [ ]:
T = transit_matrix(train.open_channels)

In [ ]:
np.round(T,3)

In [ ]:
plt.figure(figsize=(11,11))
sns.heatmap( T,annot=True, fmt='.3f', cmap='Reds', vmin=0, vmax=0.5, linewidths=3)
plt.show()

전체 tain데이터를 대상으로 구한 transit probability 를 보면, 주로 현재 오픈채널 기준대비 -2 ~ +2 정도 범위의 변화를 보임

해당 전이확률은 다음 채널수를 예측하는데 참고용

In [ ]:
eig_values, eig_vectors = np.linalg.eig(np.transpose(T))
print("Eigenvalues :", eig_values)

In [ ]:
np.round(eig_vectors,3) # console출력이 길게 나오게하는 옵션이 있는지?? 중간에 짤리니 보기 불편함 

In [ ]:
T1 = transit_matrix(batch[1].open_channels)

In [ ]:
plt.figure(figsize=(11,11))
sns.heatmap( T1,annot=True, fmt='.3f', cmap='Reds', vmin=0, vmax=0.5, linewidths=3)
plt.show()

In [ ]:
eig_values, eig_vectors = np.linalg.eig(np.transpose(T1))
print("Eigen values :", eig_values)
print("Eigen vectors :\n", np.round(eig_vectors,3))

아이겐밸류(고유값) 분해: 특성방정식, 아이겐밸류, 아이겐벡터 참고

[https://datascienceschool.net/view-notebook/7fd58178d9e64be682058db7e024d8b5/](https://datascienceschool.net/view-notebook/7fd58178d9e64be682058db7e024d8b5/)



In [ ]:
dist01 = eig_vectors[:,0] / np.sum(eig_vectors[:,0])
print("Probability distribution for sequence 3 :", dist01)

In [ ]:
np.histogram(batch[1].open_channels, bins=[0,1,2,3], density=True)[0]

# 위의 아이겐 벡터와 아래 0번,1번 채널 오픈수의 확률값이 같은 이유는?
일단 전이확률이 행렬이 A, 아이겐벡터 v,아이겐밸류 𝜆 = 1 

### Av = 𝜆v
 v 값이 각 상태에 수렴된 확률값이니~ 같은 거라고 생각함. 
 
 또 아이겐밸류 𝜆 는 1 기준이니... 동일 스케일로 비교 됨   




신경망 행렬계산 참고: https://cs.stanford.edu/people/karpathy/convnetjs/

: 선형변환 + 비선형변환



In [ ]:
def create_axes_grid(numplots_x, numplots_y, plotsize_x=6, plotsize_y=3):
    fig, axes = plt.subplots(numplots_y, numplots_x)
    fig.set_size_inches(plotsize_x * numplots_x, plotsize_y * numplots_y)
    fig.subplots_adjust(wspace=0.05, hspace=0.05)
    return fig, axes
    
def set_axes(axes, use_grid=True, x_val = [0,100,10,5], y_val = [-50,50,10,5]):
    axes.grid(use_grid)
    axes.tick_params(which='both', direction='inout', top=True, right=True, labelbottom=True, labelleft=True)
    axes.set_xlim(x_val[0], x_val[1])
    axes.set_ylim(y_val[0], y_val[1])
    axes.set_xticks(np.linspace(x_val[0], x_val[1], np.around((x_val[1] - x_val[0]) / x_val[2] + 1).astype(int)))
    axes.set_xticks(np.linspace(x_val[0], x_val[1], np.around((x_val[1] - x_val[0]) / x_val[3] + 1).astype(int)), minor=True)
    axes.set_yticks(np.linspace(y_val[0], y_val[1], np.around((y_val[1] - y_val[0]) / y_val[2] + 1).astype(int)))
    axes.set_yticks(np.linspace(y_val[0], y_val[1], np.around((y_val[1] - y_val[0]) / y_val[3] + 1).astype(int)), minor=True)
    

In [ ]:
data = batch[5].signal

fig, axes = create_axes_grid(1,1,10,10)
set_axes(axes, x_val=[-4,8,1,.1], y_val=[-4,8,1,.1])

axes.set_aspect('equal')
axes.scatter(np.roll(data,-1), data, s=.01);

In [ ]:
data = batch[5].signal
data_true = batch[5].open_channels

fig, axes = create_axes_grid(1,1,10,10)
set_axes(axes, x_val=[-4,8,1,.1], y_val=[-4,8,1,.1])

axes.set_aspect('equal')
for i in range(11):
    axes.scatter(np.roll(data,-1)[data_true==i], data[data_true==i], s=.01);

일단  해석의 어려움이 있음....

마코프체인과 노이즈(가우시안분포, 화이트노이즈)를 추가해서 생성해서 훈련데이터를 만들 수 있다....

원 저자의 해석내용을 이해하려면 관련 지식이 더 필요함...



푸리에변환과 웨이블릿변환 참고
[https://bskyvision.com/404](https://bskyvision.com/404)

![비교](https://t1.daumcdn.net/cfile/tistory/99B4BF495C41525405)

시계열 분해

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
train.head()

In [ ]:
decompose_01 = seasonal_decompose(batch[1].signal, model='additive', period=1)

In [ ]:
decompose_01.plot()

In [ ]:
def plot_decompose(result):
    fig, (ax1,ax2,ax3,ax4) = plt.subplots(4,1, figsize=(20,10))
    result.observed.plot(legend=False, ax=ax1)
    ax1.set_ylabel('Observed')
    result.trend.plot(legend=False, ax=ax2)
    ax2.set_ylabel('Trend')
    result.seasonal.plot(legend=False, ax=ax3)
    ax3.set_ylabel('Seasonal')
    result.resid.plot(legend=False, ax=ax4)
    ax4.set_ylabel('Resid')

In [ ]:
plot_decompose(decompose_01)

In [ ]:
decompose = []
decompose.append([])
for i in range(10):
    decompose.append(seasonal_decompose(batch[i+1].signal, model='additive', period=1))

In [ ]:
plot_decompose(decompose[1])

In [ ]:
plot_decompose(decompose[2])

In [ ]:
plot_decompose(decompose[3])

In [ ]:
plot_decompose(decompose[4])

In [ ]:
plot_decompose(decompose[5])

In [ ]:
plot_decompose(decompose[6])

# 데이터의 구성
참고 https://www.kaggle.com/cdeotte/rapids-knn-30-seconds-0-938

*data = computer generated + real life noise + synthetic drift*  <-- 논문참조 필요

We will use the Kaggle dataset [here][1] containing this competition's train and test data with drift removed. 

Drift is explained [here][2]. And visualized [here][3]. 

This competition's data is believed to be `data = computer generated + real life noise + synthetic drift`. 

Drift was found in training data batches 2, 7, 8, 9, 10. And test batches 1, 2, 3. 

[Markus][5] demonstrated in his great notebook [here][4] that drift can be cleanly removed with 4th order approximation functions. 

=> drift제거 부분은 별도로 따로 설명필요 함


Clean data allows to produce better EDA and allows us to use models like kNN.

[1]: https://www.kaggle.com/cdeotte/data-without-drift
[2]: https://www.kaggle.com/c/liverpool-ion-switching/discussion/133874
[3]: https://www.kaggle.com/cdeotte/one-feature-model-0-930
[4]: https://www.kaggle.com/friedchips/clean-removal-of-data-drift
[5]: https://www.kaggle.com/friedchips




In [ ]:
train_clean = pd.read_csv('../input/ion-channel-without-drift/train_clean.csv')
test_clean = pd.read_csv('../input/ion-channel-without-drift/train_clean.csv')

In [ ]:
plt.figure(figsize=(20,5)); res = 1000 #간격을 1000 만큼씩 띄어서 표시
x = range(0,train.shape[0],res)
y = train.signal[0::res]
plt.plot(x,y,'b',alpha=0.7)
for i in range(11): 
    plt.plot([i*500000,i*500000],[-5,12.5],'r')
for j in range(10): 
    plt.text(j*500000+200000,10,str(j+1),size=16)
plt.xlabel('Row',size=16); plt.ylabel('Signal & Ion channels',size=16); 
plt.title('Training Data Signal - 10 batches',size=20)

#plt.figure(figsize=(20,5))
y2 = train.open_channels[0::res]
plt.plot(x,y2,'r',alpha=0.3) #Ion 채널 수 표시

plt.show()

Drift가 있는 부분: 2, 7, 8, 9, 10  

In [ ]:
plt.figure(figsize=(20,5)); res = 1000 #간격을 1000 만큼씩 띄어서 표시
x = range(0,train_clean.shape[0],res)
y = train_clean.signal[0::res]
plt.plot(x,y,'black',alpha=0.7)
for i in range(11): 
    plt.plot([i*500000,i*500000],[-5,12.5],'r')
for j in range(10): 
    plt.text(j*500000+200000,10,str(j+1),size=16)
plt.xlabel('Row',size=16); plt.ylabel('Signal & Ion channels',size=16); 
plt.title('Training Data Signal without drift - 10 batches',size=20)

#plt.figure(figsize=(20,5))
y2 = train_clean.open_channels[0::res]
plt.plot(x,y2,'r',alpha=0.3) #Ion 채널 수 표시

plt.show()

In [ ]:
train_clean['group'] = -1
x = [(0,500000),        # 0
     (1000000,1500000), # 1
     (1500000,2000000), # 2
     (2500000,3000000), # 3
     (2000000,2500000)  # 4 
    ]

for k in range(5): 
    train_clean.iloc[x[k][0]:x[k][1],3] = k
    
res = 1000
plt.figure(figsize=(20,5))
plt.plot(train_clean.time[::res],train_clean.signal[::res], color='gray')
plt.plot(train_clean.time,train_clean.group,color='blue')
plt.title('Clean Train Data. Blue line is signal. Black line is group number.')
plt.xlabel('time'); plt.ylabel('signal')
plt.show()

In [ ]:
from scipy.stats import mode


In [ ]:
step = 0.2
pt = [[],[],[],[],[]]
cuts = [[],[],[],[],[]]
for g in range(5):
    mn = train_clean.loc[train_clean.group==g].signal.min()
    mx = train_clean.loc[train_clean.group==g].signal.max()
    old = 0
    for x in np.arange(mn,mx+step,step):
        sg = train_clean.loc[(train_clean.group==g)&(train_clean.signal>x-step/2)&(train_clean.signal<x+step/2)].open_channels.values
        if len(sg)>100:
            m = mode(sg)[0][0]
            pt[g].append((x,m))
            if m!=old: cuts[g].append(x-step/2)
            old = m
    pt[g] = np.vstack(pt[g])
    
models = ['1 channel low prob',
          '1 channel high prob',
          '3 channel',
          '5 channel',
          '10 channel']
plt.figure(figsize=(15,8))
for g in range(5):
    plt.plot(pt[g][:,0],pt[g][:,1],'-o',label='Group %i (%s model)'%(g,models[g]))
plt.legend()
plt.title('Traing Data Open Channels versus Clean Signal Value',size=16)
plt.xlabel('Clean Signal Value',size=16)
plt.ylabel('Open Channels',size=16)
plt.show()